# Load The Dataset

The first step is to load the dataset we'll use to train the classifier.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

In [3]:
print(iris['DESCR'])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

As the petal width and length are highly correlated with the target classes (the labels), we'll use these 2 features for the classifier.

In [4]:
X = iris['data'][:,(2,3)] # Petal length and width
y = iris['target']

Let's split this into the training and test set now, with an 80/20 split:

In [5]:
len(y)

150

In [6]:
from sklearn.model_selection import train_test_split

X_train_init, X_test_init, y_train_init, y_test_init = train_test_split(X,y, test_size=0.2,random_state=42)

# Compute A Benchmark

We'll use Scikit-learn's Softmax Regression model with regularization as a bench mark for this model, with an $\ell_2$ penalty and $C=\frac{1}{\alpha}=10$. First we'll determine the training time and then compute the accuracy on the test set.

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
softmax_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=10, penalty='l2')

In [9]:
softmax_reg.fit(X_train_init,y_train_init)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
softmax_reg_timed = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=10, penalty='l2')

In [11]:
%timeit softmax_reg_timed.fit(X_train_init,y_train_init)

9.5 ms ± 630 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's check the accuracy:

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
y_bench_predicts = softmax_reg.predict(X_test_init)

In [14]:
accuracy_score(y_test_init,y_bench_predicts)

1.0

So Scikit-learn's implementation takes 9.5 ms to train and has an accuracy of approximately 100%. While I'd normally consider this to be massively overfitting the test set, in this case it has perfect accuracy due to the very small size of the dataset (test set has only 30 instances).

Let's see how close we can get to these metrics using numpy.

# Custom Implementation

So the goal of this is to implement Batch Gradient Descent with early stopping for a Softmax Regression model without using Scikit-learn. As we're using Batch GD, the algorithm will more than likely take longer to train than Scikit-learn's implementation, however it should eventually converge to the optimal parameters. The first step is to create a training and test set.

## Create a training and test set

First we'll randomly shuffle the X and y indices and then use an 80/20 split for the training/test set.

In [15]:
def split_train_test_set(X,y, test_size=0.2):
    # This function creates a train and test set from the data provided using the test set size ratio provided.
    # X and y are assumed to be numpy arrays.
    rng = np.random.RandomState(42)
    shuffled_indices = rng.permutation(len(y))
    split_index = np.int_(test_size*len(y))
    train_indices = shuffled_indices[split_index:]
    test_indices = shuffled_indices[:split_index]
    return X[train_indices],y[train_indices], X[test_indices], y[test_indices]

In [16]:
X_train, y_train, X_test, y_test = split_train_test_set(X,y)

In [17]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(120, 2) (120,) (30, 2) (30,)


## Creating the Softmax Regression class

The class needs to have 2 methods: fit and predict. Initializing the model will have 3 parameters: the penalty to apply (l2 by default), the regularization parameter alpha, and the learning rate eta.

The model uses Batch Gradient Descent with early stopping to train and tune the parameters.

As the model uses early stopping, the fit method will split the training data provided into a smaller training and validation set, which assumes that there was an 80/20 split for the test set. This means the ratio for the training/validation split is 75/25. A column of 1s is concatenated to the  initial training set before being split.

The fit method then calls the predict method to get an array of predictions, which it then uses to compute the gradients. After each gradient descent step, the accuracy on the validation step is computed - if it is smaller than all the previous scores, it is saved. The gradient descent algorithm stops once the validation accuracy has consistently decreased for the last 100 epochs. The model paramaters are then rolled back to the last 'best' parameters. These are the parameters used for the model's predictions.

In [18]:
class SoftmaxRegression():
    def __init__(self, penalty='l2', alpha=0.1, eta0 = 0.1):
        if not(penalty in ['l2', None]):
            # This class will only specify an l2 regularization (half the square of the norm of the weights)
            raise Exception('Unrecognized penalty. Only \'l2\' or None penalty is specified for this class.')
        else:
            self.penalty = penalty
            self.alpha = alpha
            self.eta0 = eta0
            print('SoftmaxRegression(penalty={}, alpha={}, eta0={})'.format(penalty, alpha, eta0))
            return None
        
    def fit(self, X, y, iters = 1000):
        # The model will use Batch GD with early stopping to fit the model. Therefore the training
        # data needs to be split into a validation set internally for early stopping.
        # A column of 1s also needs to be appended to the set, which is carried out in function
        # __split_train_val_set:
        X_train,y_train, X_val, y_val = self.__split_train_val_set(X,y)        
        self.K = len(np.unique(y_train)) # No. of classes.
        self.n = np.size(X_train,axis=1) # No. of features.
        self.m = np.size(X_train,axis=0) # No. of training instances.    
        # Randomly initialize k x n array of theta values:
        self.Theta = np.random.randn(self.K, self.n)     
        y_train_prep, y_val_prep = self.__one_hot(y_train), self.__one_hot(y_val) # Encode labels using OneHotEncoding.     
        if self.penalty == None:
            # Rather than 2 large if/else blocks with much the same code (only difference being regularization term),
            # we simply set alpha to 0 if no regularization is desired, which would set the regularization term to 0.
            self.alpha = 0
        # Initialise variables for GD and early stopping:
        min_val_score = 0
        best_theta = np.zeros_like(self.Theta)
        gradients = np.zeros_like(self.Theta)
        epoch_since_min = 0           
        for epoch in range(iters):
            if epoch_since_min >= 100:
                # Breaks the loop if the validation error has increased for the last 100 training epochs.
                break
            feature_weights = np.copy(self.Theta) # Weights vector excluding bias term for regularization.
            feature_weights[:,0] = 0
            p_hat = self.predict(X_train,predict_proba=True, add_bias=False)
            error = p_hat - y_train_prep
            for i in range(self.K):
                gradients[i,:] = error[:,i].reshape(1,-1)@X_train # Gradient sum term
            gradients = gradients*(1/self.m) + self.alpha*feature_weights
            self.Theta = self.Theta - self.eta0*gradients # Gradient descent step              
            y_val_predicts = self.predict(X_val, add_bias=False) # Predictions on validation set.
            score = self.__accuracy(y_val,y_val_predicts) # Compute accuracy on validation set.              
            if score > min_val_score: # If the new theta values give a better accuracy on the validation set.
                best_theta = self.Theta
                min_val_score = score
                epoch_since_min = 0         
            else:
                # Keeps count of the number of training epochs since the last best validation score was found.
                epoch_since_min += 1
        self.Theta = best_theta # The model parameters that gave the best accuracy on the validation set are used.
                  
    def predict(self,X, one_hot = False, predict_proba = False, add_bias = True):
        # The purpose of this function is to make a prediction on an instance X using the parameters Theta. 
        # The optional argument one_hot=False returns an array of labels corresponding to the class index
        # k i.e. for 3 classes it'll return one of [0,1,2]. If one_hot = True, it'll return a OneHot encoded
        # version of the labels.
        # The optional argument predict_proba allows the array of predicted probabilities (p_hat) to be
        # returned from the function.
        # Optional argument add_bias determines whether to concatenate a column of 1s to the X array for 
        # the bias term.
        if one_hot and predict_proba:
            # Both of these options can't be true as it will always return p_hat first and quit
            # the function.
            raise Exception('Can not return one hot encoding of labels and prediction probabilities simultaneously.')
        else:
            pass
        if X.ndim == 1:
            # In case a single instance is passed to the function.
            X = X.reshape(1,-1)
        else:
            pass
        if add_bias:
            X_prep = np.c_[np.ones((np.size(X,axis=0),1)),X]
        else:
            X_prep = X
        softmax_score = X_prep.dot(self.Theta.T)
        exponents = np.exp(softmax_score)
        p_hat = np.divide(exponents, np.sum(exponents, axis=1).reshape(-1,1))        
        if predict_proba:
            return p_hat
        else:    
            predictions = np.argmax(p_hat,axis=1)
            if one_hot:
                return self.__one_hot(predictions.ravel())       
            else:
                return predictions.ravel()
        
    def __split_train_val_set(self, X, y, val_size=0.25):
        # This is a private method that creates a train and validation set from 
        # the data provided using a 75/25 split. X and y are assumed to be numpy arrays.
        X_prep = np.c_[np.ones((np.size(X,axis=0),1)),X] # Concatenate bias term to data.
        shuffled_indices = np.random.permutation(len(y))
        split_index = np.int_(val_size*len(y))
        train_indices = shuffled_indices[split_index:]
        val_indices = shuffled_indices[:split_index]   
        return X_prep[train_indices],y[train_indices], X_prep[val_indices], y[val_indices]
    
    def __one_hot(self,labels):
        # This private method is a custom implementation for OneHotEncoder that will convert the labels array
        # of k values (class indexes) into a one hot encoded matrix.
        encoded = np.zeros((len(labels),self.K))
        encoded[np.arange(len(labels)), labels] = 1
        return encoded
    
    def __accuracy(self,labels,predictions):
        # Computes the accuracy to evaluate generalization score on validation set.
        if labels.shape != predictions.shape:
            raise Exception('Error: Label and prediction dimensions do not match')
        else:
            correct = np.sum(labels==predictions,axis=0)
            return correct/(np.size(labels,axis=0))

## How does it perform?

Now that the SoftmaxRegression class is finished, how does it perform compared to Scikit-learn's version?

In [19]:
soft_reg = SoftmaxRegression(penalty=None) # No regularization.

SoftmaxRegression(penalty=None, alpha=0.1, eta0=0.1)


In [20]:
soft_reg.fit(X_train,y_train)

In [21]:
def accuracy(labels,predictions):
    correct = np.sum(labels==predictions,axis=0)
    return correct/(np.size(labels,axis=0))

In [22]:
test_predictions = soft_reg.predict(X_test)

In [23]:
accuracy(y_test,test_predictions)

0.9

In [24]:
soft_reg_l2 = SoftmaxRegression() # With l2 regularization.

SoftmaxRegression(penalty=l2, alpha=0.1, eta0=0.1)


In [47]:
soft_reg_l2.fit(X_train,y_train)

In [48]:
test_predictions_l2 = soft_reg_l2.predict(X_test)

In [49]:
accuracy(y_test,test_predictions_l2)

0.8

What about the time taken to train?

In [28]:
timed_clf = SoftmaxRegression()

SoftmaxRegression(penalty=l2, alpha=0.1, eta0=0.1)


In [29]:
%timeit timed_clf.fit(X_train,y_train)

20.6 ms ± 4.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


So, this implementation takes twice as long as Scikit-learn's. Not bad considering there is no optimization done and it's all written in Python.

From running the code multiple times, it's hard to effectively determine the performance of the classifier with such a small dataset. The runtime varies from 12 ms to 23 ms, with the accuracy on the test set varying anywhere from 30% to 97%. A larger dataset is needed to determine a more consistent performance.

## Performance on MNIST dataset

Fortunately, the MNIST dataset is much larger and should give a much better estimate of the model's performance.

In [30]:
from sklearn.datasets import fetch_openml

In [31]:
mnist = fetch_openml('mnist_784', version=1)

In [32]:
X_mn, y_mn = mnist['data'], mnist['target']

In [33]:
y_mn = y_mn.astype(int) # Labels are strings.

In [34]:
X_train_mn, y_train_mn, X_test_mn, y_test_mn = split_train_test_set(X_mn,y_mn)

We need to scale the features of the dataset before training. For convenience I'm going to cheat and use sklearn's StandardScaler:

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
scaler = StandardScaler()

In [37]:
X_train_mn = scaler.fit_transform(X_train_mn)

In [38]:
X_test_mn = scaler.transform(X_test_mn)

In [39]:
mnist_clf = SoftmaxRegression()

SoftmaxRegression(penalty=l2, alpha=0.1, eta0=0.1)


In [40]:
mnist_clf.fit(X_train_mn[:10000], y_train_mn[:10000], iters=2000) # 2000 iterations due to the size of the dataset.

In [41]:
predictions = mnist_clf.predict(X_test_mn[:2000])

In [42]:
accuracy(y_test_mn[:2000],predictions)

0.896

89.6% accuracy. Not bad. What about Scikit-learn's model?

In [43]:
mnist_clf_skl = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=10, penalty='l2', max_iter = 2000)

In [44]:
mnist_clf_skl.fit(X_train_mn[:10000], y_train_mn[:10000])

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
skl_predictions = mnist_clf_skl.predict(X_test_mn[:2000])

In [46]:
accuracy(y_test_mn[:2000], skl_predictions)

0.876

In this case, Scikit-learn's model actually performs worse (by 2%). I'll take that as a win.